### Dacon 
* https://www.dacon.io/competitions/official/235647/overview/description
- Preprocessing
- Classification & Regression Model
- Cross Validation + OOF Ensemble
- Submit


    
    
        예제 코드)<br>
        model = XGBClassifier(n_estimators=20000,     # Early Stopping 기능을 사용하기 때문에 높게 줍니다.
                              random_state= ,         # 원하는 숫자 넣으세요.
                              tree_method='gpu_hist', # GPU 연산 옵션
                              n_jobs= ,               # CPU 개수 - 1로 입력하세요.
                              )
        model.fit(x_train, y_train,
                  eval_set = [[x_valid, y_valid]], # eval_metric으로 평가할 평가 데이터 입니다.
                  eval_metric = ,                  # 평가에 맞는 메트릭 함수 ex) log_loss, f1_score
                  early_stopping_rounds = ,        # 얼마나 기다릴지에 대한 횟수, 주로 500, 1000 정도합니다.
                  verbose = 100,                   # 100번에 한번 씩 결과를 출력해줍니다.
                  )
                  

## 월간 데이콘 8회

In [204]:
from os.path import join


import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


#### 1. 전처리
- 과제 전처리 코드


In [236]:
%ls data/MDC08/

sample_submission.csv  test.csv               train.csv


In [175]:
data_path = join("data", "MDC08")
# data_path

data = pd.read_csv(join(data_path, "train.csv"))
test = pd.read_csv(join(data_path, "test.csv"))

target_column = list(set(data.columns) - set(test.columns))
target = data[target_column[0]].copy()

del data["index"], test["index"], data[target_column[0]]



print(data.shape)
print(target.shape)
print(test.shape)

(45532, 76)
(45532,)
(11383, 76)


In [176]:
pd.options.display.max_columns = None
data.head()

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,3.0,363,4.0,1370,5.0,997,1.0,1024,2.0,1577,5.0,539,2.0,586,4.0,1095,5.0,1142,4.0,1287,4.0,883,4.0,851,2.0,851,5.0,816,2.0,579,2.0,924,2.0,366,2.0,876,2.0,633,1.0,1115,30s,2,1,4,Female,1,3,White,Other,2,2,2,1,2,1,7,4,4,3,1,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1
1,5.0,647,5.0,1313,3.0,3387,5.0,2969,1.0,4320,3.0,2190,1.0,826,1.0,4082,5.0,1867,3.0,1264,5.0,2943,4.0,3927,1.0,4329,5.0,1828,1.0,1214,5.0,2414,5.0,1356,1.0,3039,4.0,4304,1.0,1346,20s,4,2,3,Female,1,1,Asian,Hindu,1,1,0,0,1,2,3,4,0,4,3,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,1
2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,1.0,531,4.0,1167,1.0,1016,3.0,2653,2.0,1569,5.0,998,5.0,2547,2.0,918,4.0,2153,2.0,1304,1.0,1131,5.0,937,4.0,1327,1.0,1170,1.0,1409,30s,3,1,3,Male,1,2,White,Other,2,3,1,5,3,4,2,6,1,3,2,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1
3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,2.0,1519,4.0,159,3.0,2275,5.0,2809,4.0,5614,3.0,3219,4.0,1296,4.0,9046,4.0,1216,4.0,1169,4.0,23868,3.0,581,4.0,8830,4.0,2392,5.0,1312,20s,4,2,0,Female,1,1,Asian,Hindu,2,4,1,1,1,3,1,3,1,3,3,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1
4,1.0,927,1.0,707,5.0,556,2.0,1062,1.0,1014,2.0,628,1.0,991,1.0,1259,5.0,1153,5.0,1388,5.0,740,5.0,1181,4.0,547,2.0,575,1.0,754,4.0,1140,5.0,323,5.0,1070,1.0,583,2.0,1889,20s,3,1,2,Male,1,2,White,Agnostic,1,1,1,6,0,2,0,6,2,6,1,0,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1


In [177]:
test.head()

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,3.0,736,2.0,2941,3.0,4621,1.0,4857,2.0,2550,4.0,3171,5.0,3613,2.0,1494,4.0,1597,2.0,8891,5.0,2416,3.0,2043,3.0,3445,5.0,1232,3.0,1152,4.0,1447,3.0,798,4.0,1054,1.0,3462,2.0,1621,10s,2,2,3,Male,1,1,White,Christian_Protestant,2,1,1,4,1,2,2,3,4,4,2,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,1
1,3.0,514,2.0,1952,3.0,1552,3.0,821,4.0,1150,1.0,605,2.0,601,1.0,2370,5.0,2268,3.0,1759,5.0,771,5.0,1124,3.0,4525,1.0,559,2.0,988,3.0,1014,4.0,380,5.0,995,3.0,629,4.0,708,10s,2,2,2,Male,2,1,Other,Christian_Catholic,7,7,7,7,7,7,7,7,7,7,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3.0,500,2.0,2507,4.0,480,2.0,614,2.0,1326,2.0,767,1.0,799,1.0,1309,2.0,3317,4.0,766,4.0,1940,5.0,926,4.0,568,2.0,608,5.0,669,3.0,2417,3.0,710,4.0,2781,3.0,1110,5.0,1073,30s,3,2,7,Male,1,2,Other,Christian_Other,6,2,3,0,6,1,2,2,4,1,1,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1
3,1.0,669,1.0,1050,5.0,1435,2.0,2252,5.0,2533,1.0,578,1.0,742,1.0,2886,1.0,1379,1.0,1392,3.0,2855,5.0,697,4.0,573,1.0,412,2.0,1076,4.0,1445,5.0,1069,1.0,847,4.0,1126,1.0,1240,60s,4,1,4,Male,1,2,White,Christian_Catholic,0,5,1,6,0,6,0,5,1,6,2,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1
4,2.0,499,1.0,1243,5.0,845,2.0,1666,2.0,925,2.0,413,5.0,361,1.0,995,2.0,1427,3.0,1093,5.0,1201,4.0,1264,2.0,943,4.0,475,2.0,787,4.0,607,3.0,529,4.0,947,4.0,524,3.0,1945,10s,2,1,3,Male,1,1,White,Agnostic,6,1,2,2,3,0,5,2,4,3,2,0,0,0,1,1,0,1,1,0,1,1,0,1,1,1,1


-----

##### data: dtypes: float64(20), int64(52), object(4)<br>test: dtypes: float64(20), int64(52), object(4)


In [178]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45532 entries, 0 to 45531
Data columns (total 76 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   QaA         45532 non-null  float64
 1   QaE         45532 non-null  int64  
 2   QbA         45532 non-null  float64
 3   QbE         45532 non-null  int64  
 4   QcA         45532 non-null  float64
 5   QcE         45532 non-null  int64  
 6   QdA         45532 non-null  float64
 7   QdE         45532 non-null  int64  
 8   QeA         45532 non-null  float64
 9   QeE         45532 non-null  int64  
 10  QfA         45532 non-null  float64
 11  QfE         45532 non-null  int64  
 12  QgA         45532 non-null  float64
 13  QgE         45532 non-null  int64  
 14  QhA         45532 non-null  float64
 15  QhE         45532 non-null  int64  
 16  QiA         45532 non-null  float64
 17  QiE         45532 non-null  int64  
 18  QjA         45532 non-null  float64
 19  QjE         45532 non-nul

In [179]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11383 entries, 0 to 11382
Data columns (total 76 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   QaA         11383 non-null  float64
 1   QaE         11383 non-null  int64  
 2   QbA         11383 non-null  float64
 3   QbE         11383 non-null  int64  
 4   QcA         11383 non-null  float64
 5   QcE         11383 non-null  int64  
 6   QdA         11383 non-null  float64
 7   QdE         11383 non-null  int64  
 8   QeA         11383 non-null  float64
 9   QeE         11383 non-null  int64  
 10  QfA         11383 non-null  float64
 11  QfE         11383 non-null  int64  
 12  QgA         11383 non-null  float64
 13  QgE         11383 non-null  int64  
 14  QhA         11383 non-null  float64
 15  QhE         11383 non-null  int64  
 16  QiA         11383 non-null  float64
 17  QiE         11383 non-null  int64  
 18  QjA         11383 non-null  float64
 19  QjE         11383 non-nul

In [180]:
data.describe()

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,education,engnat,familysize,hand,married,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
count,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,45532.000000,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,45532.000000,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,4.553200e+04,45532.000000,45532.000000,4.553200e+04,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000,45532.000000
mean,2.129535,9.453570e+02,2.904463,2.189589e+03,3.662347,1484.294518,1.749078,1.490672e+03,2.317952,1.899292e+03,2.168145,1.850650e+03,2.560595,1.284608e+03,2.317952,4.584272e+03,3.157691,4.068434e+03,3.575529,2.576363e+03,3.813933,2.038986e+03,4.325727,3.741018e+03,3.509620,2.050287e+03,2.632896,959.880502,3.217232,1.435762e+03,3.071752,1.633479e+03,2.840464,1.154352e+03,2.823487,2.470808e+03,3.432948,1.164117e+03,3.225687,2.240057e+03,2.544474,1.271326,4.716689e+04,1.170715,1.387112,3.017241,2.621189,1.768712,3.041246,1.335918,2.472217,2.031099,3.507226,2.277497,4.317996,2.180401,0.079592,0.163665,0.118356,0.709721,0.934793,0.208974,0.868708,0.895722,0.328538,0.938900,0.972195,0.202824,0.946104,0.395195,0.947729,0.927040
std,1.196952,1.307565e+04,1.566142,3.351027e+04,1.431494,8977.664318,1.043625,1.092260e+04,1.369205,1.670765e+04,1.348653,7.608236e+04,1.510600,9.756584e+03,1.460813,5.020265e+05,1.481740,1.141738e+05,1.361220,5.411432e+04,1.372551,3.069164e+04,1.146992,3.663672e+05,1.375134,7.530787e+04,1.473022,5869.964127,1.476414,1.084997e+04,1.489744,2.600221e+04,1.404342,2.250689e+04,1.424383,1.482890e+05,1.374938,7.433515e+03,1.521603,6.730924e+04,0.941328,0.448436,1.006402e+07,0.468883,0.639987,2.007713,1.897081,1.685069,2.037278,1.452544,2.000658,1.785824,1.971456,1.848783,1.722793,0.762647,0.270664,0.369975,0.323033,0.453896,0.246893,0.406580,0.337723,0.305625,0.469687,0.239516,0.164414,0.402107,0.225815,0.488898,0.222575,0.260073
min,1.000000,2.500000e+01,1.000000,2.500000e+01,1.000000,25.000000,1.000000,2.600000e+01,1.000000,2.500000e+01,1.000000,2.500000e+01,1.000000,2.500000e+01,1.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,2.500000e+01,1.000000,2.500000e+01,1.000000,25.000000,1.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,2.500000e+01,1.000000,2.500000e+01,1.000000,2.500000e+01,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,4.040000e+02,1.000000,8.750000e+02,2.000000,651.000000,1.000000,6.790000e+02,1.000000,8.340000e+02,1.000000,5.040000e+02,1.000000,5.490000e+02,1.000000,1.077000e+03,2.000000,1.259000e+03,2.000000,9.020000e+02,3.000000,8.590000e+02,4.000000,8.060000e+02,2.000000,6.500000e+02,1.000000,428.000000,2.000000,6.380000e+02,2.000000,7.060000e+02,2.000000,4.610000e+02,2.000000,7.940000e+02,2.000000,5.270000e+02,2.000000,7.330000e+02,2.000000,1.000000,2.000000e+00,1.000000,1.000000,1.000000,1.000000

In [181]:
test.describe()

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,education,engnat,familysize,hand,married,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
count,11383.000000,11383.000000,11383.000000,1.138300e+04,11383.000000,1.138300e+04,11383.000000,1.138300e+04,11383.000000,1.138300e+04,11383.000000,11383.000000,11383.000000,1.138300e+04,11383.000000,11383.000000,11383.000000,1.138300e+04,11383.000000,1.138300e+04,11383.000000,1.138300e+04,11383.000000,1.138300e+04,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,1.138300e+04,11383.000000,1.138300e+04,11383.000000,1.138300e+04,11383.000000,1.138300e+04,11383.000000,11383.000000,11383.000000,1.138300e+04,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.000000,11383.00000,11383.000000
mean,2.094351,825.556268,2.930159,2.375705e+03,3.665027,4.627016e+03,1.733199,1.573476e+03,2.322498,1.908333e+03,2.183607,1261.964069,2.551876,1.503954e+03,2.323201,2023.762365,3.163577,4.501748e+03,3.582799,2.626536e+03,3.814197,2.463975e+03,4.328648,1.971527e+03,3.524466,1449.079505,2.635070,905.620135,3.242379,1.695724e+03,3.079856,1.705669e+03,2.804533,1.135987e+03,2.839497,2.715532e+03,3.435913,1299.238953,3.229553,1.709446e+03,2.546077,1.270140,2.579021,1.162962,1.382764,2.993411,2.613898,1.778090,3.028903,1.351050,2.449881,2.038830,3.519722,2.295616,4.342528,2.171484,0.079856,0.167091,0.118598,0.712905,0.932970,0.208205,0.873671,0.896424,0.328824,0.938329,0.972239,0.197487,0.946148,0.396117,0.95098,0.924976
std,1.182765,2450.121716,1.557470,5.109158e+04,1.433764,3.404012e+05,1.024850,1.932751e+04,1.368660,1.647452e+04,1.357935,9516.116563,1.506418,1.969331e+04,1.467804,4494.374430,1.477513,1.258543e+05,1.354224,3.628162e+04,1.371511,5.730667e+04,1.157722,2.108026e+04,1.364862,5795.989823,1.475629,3159.081388,1.472535,2.292899e+04,1.486849,1.699703e+04,1.402425,1.450249e+04,1.426454,8.391658e+04,1.366987,11021.057030,1.515665,1.394429e+04,0.937681,0.447402,1.426420,0.456575,0.639947,1.992088,1.905473,1.677511,2.035088,1.455218,1.983041,1.809327,1.974297,1.841607,1.708390,0.771759,0.271082,0.373074,0.323329,0.452426,0.250085,0.406042,0.332234,0.304722,0.469806,0.240567,0.164294,0.398121,0.225736,0.489111,0.21592,0.263442
min,1.000000,26.000000,1.000000,2.900000e+01,1.000000,2.700000e+01,1.000000,2.800000e+01,1.000000,2.900000e+01,1.000000,0.000000,1.000000,2.700000e+01,1.000000,26.000000,1.000000,3.400000e+01,1.000000,2.600000e+01,1.000000,2.500000e+01,1.000000,2.800000e+01,1.000000,0.000000,1.000000,25.000000,1.000000,2.700000e+01,1.000000,2.800000e+01,1.000000,2.500000e+01,1.000000,3.200000e+01,1.000000,25.000000,1.000000,2.700000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,1.000000,403.000000,1.000000,8.830000e+02,2.000000,6.560000e+02,1.000000,6.710000e+02,1.000000,8.300000e+02,1.000000,504.000000,1.000000,5.500000e+02,1.000000,1087.000000,2.000000,1.249500e+03,2.000000,8.900000e+02,3.000000,8.630000e+02,4.000000,8.050000e+02,2.000000,658.000000,1.000000,430.000000,2.000000,6.350000e+02,2.000000,7.030000e+02,1.000000,4.630000e+02,2.000000,7.930000e+02,2.000000,533.000000,2.000000,7.370000e+02,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,2.0

-----

#### NA check

In [182]:
data.isna().sum().sum()

0

In [183]:
test.isna().sum().sum()

0

-----

#### category columns & numeric columns




In [184]:
print(len(data.select_dtypes(include = "object").columns.to_list()))
len(test.select_dtypes(include = "object").columns.to_list())

4


4

In [185]:
cat_columns = data.select_dtypes(include = "object").columns.to_list()
num_columns = data.select_dtypes(exclude = "object").columns.to_list()

data.shape[1] == (len(cat_columns) + len(num_columns))

True

---

In [190]:
le = LabelEncoder()

target = le.fit_transform(target)

target

array([1, 1, 0, ..., 0, 0, 0])

In [186]:
def change_cat_val(df):
    df["religion"] = df["religion"].replace({"Other" : "religion_other"})
    df["race"]= df["race"].replace({"Other" : "race_other"})
    
    return df

def preprocess(X_train, X_valid, X_test):
    X_train_tmp = X_train.copy()
    X_valid_tmp = X_valid.copy()
    X_test_tmp = X_test.copy()
    
    X_train_tmp = X_train_tmp.reset_index(drop = True)
    X_valid_tmp = X_valid_tmp.reset_index(drop = True)
    X_test_tmp = X_test_tmp.reset_index(drop = True)
    
    
    #NA
#     if X_train.isna().sum().sum() != 0:

    

    # scaling
    sdscaler = StandardScaler()
    X_train_tmp[num_columns] = sdscaler.fit_transform(X_train_tmp[num_columns])
    X_valid_tmp[num_columns] = sdscaler.transform(X_valid_tmp[num_columns])
    X_test_tmp[num_columns] = sdscaler.transform(X_test_tmp[num_columns])
    
    
    # encoding
    ohe = OneHotEncoder(sparse= False)
    concat = pd.concat([X_train_tmp, X_valid_tmp, X_test_tmp], axis = 0)
    concat = change_cat_val(concat)
    ohe.fit(concat[cat_columns])
    

    X_train_tmp = change_cat_val(X_train_tmp)
    X_valid_tmp = change_cat_val(X_valid_tmp)
    X_test_tmp = change_cat_val(X_test_tmp)
    
    train_ohe = ohe.transform(X_train_tmp[cat_columns])
    valid_ohe = ohe.transform(X_valid_tmp[cat_columns])
    test_ohe = ohe.transform(X_test_tmp[cat_columns])
    
    ohe_cat = []
    for cat in ohe.categories_:
        ohe_cat += cat.tolist()
    
    
    X_train_tmp = pd.concat([X_train_tmp, pd.DataFrame(train_ohe, columns = ohe_cat)], axis = 1)
    X_valid_tmp = pd.concat([X_valid_tmp, pd.DataFrame(valid_ohe, columns = ohe_cat)], axis = 1)
    X_test_tmp = pd.concat([X_test_tmp, pd.DataFrame(test_ohe, columns = ohe_cat)], axis = 1)
    
    X_train_tmp = X_train_tmp.drop(cat_columns, axis = 1)
    X_valid_tmp = X_valid_tmp.drop(cat_columns, axis = 1)
    X_test_tmp = X_test_tmp.drop(cat_columns, axis = 1)
    
    
    
    
    
    
    
    return X_train_tmp, X_valid_tmp, X_test_tmp
    
    
    
    

#### 2. 모델 검증 + OOF 앙상블


In [227]:
from lightgbm import LGBMClassifier
n_splits = 5
skf = StratifiedKFold(n_splits= n_splits, random_state= 42, shuffle= True)
lgbm_clf = LGBMClassifier(n_jobs = -1, random_state = 42)

In [238]:
val_score = []
oof_pred = np.zeros([X_test.shape[0], le.classes_.shape[0]])


for i, (trn_idx, val_idx) in enumerate(skf.split(data, target)):
    X_train, y_train = data.iloc[trn_idx,:], target[trn_idx]
    X_valid, y_valid = data.iloc[val_idx,:], target[val_idx]
    
    
    X_train, X_valid, X_test = preprocess(X_train, X_valid, test)
    
    lgbm_clf = LGBMClassifier(n_jobs = -1, random_state = 42)
    
    lgbm_clf.fit(X_train, y_train, 
                 early_stopping_rounds= 100,
                 eval_set= [[X_train, y_train],[X_valid, y_valid]],
                 eval_metric= "auc", 
                 verbose= 100)
    
    trn_auc = roc_auc_score(y_train, lgbm_clf.predict_proba(X_train)[:,1])
    val_auc = roc_auc_score(y_valid, lgbm_clf.predict_proba(X_valid)[:,1])
    
    print("{0} Fold, Train AUC: {1:.4f}, Valid AUC: {2:.4f}".format(i, trn_auc, val_auc))
    print("---"*30)
    
    val_score.append(val_auc)
    
    oof_pred += lgbm_clf.predict_proba(X_test) / n_splits
    
#     print(oof_pred)


print("Cross Validation Score : {:.4f}".format(np.mean(val_score)))

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.847119	training's binary_logloss: 0.49688	valid_1's auc: 0.780636	valid_1's binary_logloss: 0.544066
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.847119	training's binary_logloss: 0.49688	valid_1's auc: 0.780636	valid_1's binary_logloss: 0.544066
0 Fold, Train AUC: 0.8470, Valid AUC: 0.7806
------------------------------------------------------------------------------------------
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.85092	training's binary_logloss: 0.493967	valid_1's auc: 0.765133	valid_1's binary_logloss: 0.556758
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.85092	training's binary_logloss: 0.493967	valid_1's auc: 0.765133	valid_1's binary_logloss: 0.556758
1 Fold, Train AUC: 0.8509, Valid AUC: 0.7651
------------------------------------------------------------------------------------------
Training

In [229]:
oof_pred

array([[0.35920631, 0.64079369],
       [0.09842331, 0.90157669],
       [0.53927692, 0.46072308],
       ...,
       [0.78539114, 0.21460886],
       [0.62684814, 0.37315186],
       [0.33707222, 0.66292778]])

----------